In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(url, names=names)

In [8]:
transform = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
dataset["class"] = dataset["class"].apply(lambda x: transform[x])

array([0, 1, 2], dtype=int64)

In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Класс, который в последствии добавляется в словарь для удобного выбора
class Node:
    def __init__(
        self,
        feature=None,
        threshold=None,
        childs=None,
        value=None,
    ):
        self.feature = feature
        self.threshold = threshold
        self.childs = childs
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
    


class DecisionTree:
    def __init__(self, max_depth=10, min_samples=10):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None


    def fit(self, X, y):
        self.tree = self.grow_tree(X, y)

    def calck_unic(self, a: list):
        keys = set(a)
        return {key: len(a[a == key]) for key in keys}

    def predict(self, X):
        return np.array([self.travers_tree(x, self.tree) for x in X])

    def entropy(self, y: list):
        hist = self.calck_unic(y)
        hist = {key: val / len(y) for key, val in hist.items()}
        info = -np.sum([p * np.log2(p) for p in hist.values()])

        return info

    def information_gain(self, X_column: list, y: list):
        if len(set(y)) == 1:
            return 0

        n = len(y)
        # info(T)
        parent = self.entropy(y)
        uitems = self.calck_unic(X_column)
        info_x = np.sum(
            [val / n * self.entropy(y[X_column == key]) for key, val in uitems.items()]
        )
        split_info = -np.sum(
            val / n * np.log2(val / n) for val in uitems.values() if val > 0
        )

        if split_info != 0:
            return (parent - info_x) / split_info, list(uitems.keys())
        else:
            return 0, list(uitems.keys())

    def most_common(self, y):
        labels = self.calck_unic(y)
        return max(labels, key=labels.get)

    def best_split(self, X, y):
        best_feature = None
        best_gain = -1
        uitems = []
        for i in range(X.shape[1]):
            gain, now_uitems = self.information_gain(X[:, i], y)
            if gain > best_gain:
                best_gain = gain
                best_feature = i
                uitems = now_uitems + []

        return best_feature, uitems

    def grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = len(self.calck_unic(y))

        if n_samples <= self.min_samples or depth >= self.max_depth or n_labels <= 1:
            return Node(value=self.most_common(y))

        best_feature, ukeys = self.best_split(X, y)

        # В словаре содержатся не словари, а Node По сути, словарь содержит ссылки на объекты, а нужен он для более удобной навигации.
        childs = {
            key: self.grow_tree(
                X[X[:, best_feature] == key],
                y[X[:, best_feature] == key],
                depth=depth + 1,
            )
            for key in ukeys
        }

        return Node(best_feature, childs=childs)

    def travers_tree(self, x, tree):
        if tree is None:
            return None
        
        elif tree.is_leaf_node():
            return tree.value

        return self.travers_tree(
            x,
            tree.childs.get(
                x[tree.feature], tree.childs.get(list(tree.childs.keys())[0])  #None
            ),
        )


# def preload():
    # df = pd.read_csv("agaricus-lepiota.csv")
    # y = np.array(df.pop("poison"))

    # n = df.shape[1]
    # to_del = int(n ** 0.5)
    # import random

    # for i in range(n - to_del - 1):
    #     df.pop(random.choice(df.columns))
    # x = np.array(df)
    # return x, y


def acc_pre_rec(otv, y):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(otv)):
        if otv[i] == "p" and y[i] == "p":
            tp += 1
        elif otv[i] == "p" and y[i] == "e":
            fp += 1
        elif otv[i] == "e" and y[i] == "p":
            fn += 1
        elif otv[i] == "e" and y[i] == "e":
            tn += 1

    acc = (tp + tn) / (tp + tn + fp + fn)
    pre = tp / max((tp + fp), 1)
    rec = tp / max(tp + fn, 1)
    fpr = fp / max(tn + fp, 1)

    return acc, pre, rec, fpr

# x, y = preload()
x = dataset[dataset.columns[:-1]].to_numpy()[:, :2]
y = dataset["class"].to_numpy()


clf = DecisionTree(max_depth=2)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
clf.fit(x_train, y_train)

otv = clf.predict(x_test)

print(f1_score(y_test, otv, average="micro"))


# acc, pre, rec, fpr = acc_pre_rec(y_test, otv)
# print(f"accuracy: {acc}")
# print(f"precision: {pre}")
# print(f"recall: {rec} ")

# points = np.array([[0, 0], [fpr, rec], [1, 1]])
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 7))

# plt.plot(
#     points[:, 0],
#     points[:, 1],
#     "o-r",
#     alpha=0.7,
#     label="first",
#     lw=5,
#     mec="b",
#     mew=2,
#     ms=10,
# )
# plt.fill_between(points[:, 0],
#     points[:, 1], where=None, interpolate=False, step=None)

0.6


C:\Users\artbo\AppData\Local\Temp\ipykernel_17132\991102324.py:60: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  split_info = -np.sum(


In [ ]:
import numpy as np

class LogisticRegression:
    def __init__(self, lr=0.1, num_iter=1000, fit_intercept=True, verbose=False, reg_lambda=0.01):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.reg_lambda = reg_lambda

    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)

    def __softmax(self, z):
        exp_scores = np.exp(z)
        return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        self.theta = []
        for i in range(len(np.unique(y))):
            theta_i = np.zeros(X.shape[1])
            y_i = np.array([1 if label == i else 0 for label in y])
            print(f"theta_{i} = ", theta_i)
            for j in range(self.num_iter):
                z = np.dot(X, theta_i)
                h = self.__sigmoid(z)
                gradient = np.dot(X.T, (h - y_i)) / y_i.size
                gradient += (self.reg_lambda / y_i.size) * theta_i
                theta_i -= self.lr * gradient

                # log loss
                loss = np.sum(-y_i * np.log(h) - (1 - y_i) * np.log(1 - h)) / y_i.size
                if self.verbose and j % 1000 == 0:
                    print(f"loss at iteration {j} for theta_{i}: {loss}")

            self.theta.append(theta_i)

        self.theta = np.array(self.theta)


    def predict(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        return np.argmax(self.__softmax(np.dot(X, self.theta.T)), axis=1)

    def score(self, X, y):
        self.predictions = self.predict(X)
        return np.mean(self.predictions == y)

In [82]:
class GradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X, y-self.predict(X))
            self.trees.append(tree)

    def predict(self, X):
        prev_predicts = np.zeros((X.shape[0], len(self.trees)))
        for i, tree in enumerate(self.trees):
            prev_predicts[:, i]  = tree.predict(X)
        return np.sum(prev_predicts, axis=1)

boost = GradientBoosting()

boost.fit(X=x_train, y=y_train)
otv = boost.predict(x_test)
print(f1_score(y_test, otv, average="micro"))

SyntaxError: positional argument follows keyword argument (988575764.py, line 23)